# Data cleaning

In [265]:
# Imports go here
import os
import csv
import glob
import pandas as pd
import os 
import shutil
import datetime
import geopandas as gpd
from datetime import date
from datetime import datetime
from pyspark.sql.functions import col, lit
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
import pyspark.sql.types
from pyspark.sql import Row
from shutil import copyfile
from shapely.geometry import Point
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, DateType, IntegerType, BooleanType, TimestampType, FloatType
from pyspark.sql.types import LongType, StringType, StructType, StructField
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=3g  pyspark-shell"
from pyspark.sql import SparkSession
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass
# Create a new spark session (note, the * indicates to use all available CPU cores)
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("H600 L-Group") \
    .getOrCreate()
#When dealing with RDDs, we work the sparkContext object. See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
sc=spark.sparkContext
#in local mode, you will be able to access the Spark GUI at http://localhost:4040

Spark application already started. Terminating existing application and starting new one


## Auxiliary functions

### Function: Create list 

In [53]:
def create_files_list(path, brand, list_files):
    """
    This function create the files list of specify taxi brand (brand) from the specify folder (path). 
    
    Input: the path where are the files -> /data/cleaned or data/sampled
           the name of the taxi company -> fhv, fhvfh, green, yellow
           the empty file name list in which each file will be append
    Output: number of files in the list and the list of files name.
    """  
    global nb_files
    nb_files = 0
    for file in glob.glob("%s/%s/*.csv" %(path,brand)):
        nb_files = nb_files+1
        # Save in list the files name
        list_files.append(file)
        # Order by date the file list
        list_files.sort()

    return list_files, nb_files



### Function: Remove duplicates

In [64]:
def count_duplicates(df):
    df.groupBy(df.columns)\
    .count()\
    .where(f.col('count')>1)\
    .select(f.sum('count'))\
    .show()

def drop_duplicates(df,column_list):   
    df.dropDuplicates([column_list]).show()
    df.collect()        
    return df


### Function: get columns

In [6]:
def get_columns(df, col_type="relevant"):
    cols = []
    dfcols = list(df.columns)
    if col_type == "relevant":
        subs_to_check = ['time', 'location', 'passenger','distance', 
                         'ratecode', 'fare', "longitude", "latitude"]
        for sub in subs_to_check:
            for col in dfcols:
                if sub.lower() in col.lower():
                    cols.append(col)
    
    elif col_type == "geolocation":
        subs_to_check = ["location", "longitude", "latitude"]
        for sub in subs_to_check:
            for col in dfcols:
                if sub.lower() in col.lower():
                    cols.append(col)
    return cols
        
    return list_files, nb_files

#df = pd.read_csv('data/cleaned/fhv/fhv_tripdata_2020-06.csv')
#get_columns(df,col_type="geolocation")
#
#df = pd.read_csv('data/cleaned/fhv/fhv_tripdata_2020-06.csv')
#get_columns(df,col_type="relevant")

### Function: remove nan values

In [ ]:
def remove_nan_values(df):
    df = df.replace(to_replace='None', value=np.nan).dropna()
    print(df.shape)
    df = df[(df != 0).all(1)]
    print(df.shape)
    return df

### Function: fix datatype

In [7]:
def fix_column_datatypes(df):
    date_columns = ['pickup_datetime', 'dropoff_datetime']
    numeric_columns = ['passenger_count', 'trip_distance', 'fare_amount']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)
    return df

### Function: outliers

In [87]:
def calculate_bounds(df):
    bounds = {
        c: dict(
            zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
        )
        for c,d in zip(df.columns, df.dtypes) if d[1] == "int"
    }

    for c in bounds:
        iqr = bounds[c]['q3'] - bounds[c]['q1']
        bounds[c]['min'] = bounds[c]['q1'] - (iqr * 1.5)
        bounds[c]['max'] = bounds[c]['q3'] + (iqr * 1.5)

    return bounds

In [88]:
def flag_outliers(df, id_col):
    bounds = calculate_bounds(df)
    outliers = {}

    return df.select(c, id_col,
            *[
                f.when(
                    ~f.col(c).between(bounds[c]['min'], bounds[c]['max']),
                    "yes"
                ).otherwise("no").alias(c+'_outlier')
            ]
        )

In [90]:
calculate_bounds(green_DF)
#flag_outliers(green_DF,'Tip_amount')
#green_DF.show(5)

{'pulocationid': {'q1': 50.0, 'q3': 178.0, 'min': -142.0, 'max': 370.0},
 'dolocationid': {'q1': 151.0, 'q3': 9999.0, 'min': -14621.0, 'max': 24771.0}}

In [ ]:
#?. fonction to remove useless columns and create new file (last one it creates new files)
    def drop_columns(input_file,output_file,cols_to_remove):
    """
    This function removes all columns that were considered not useful for the analysis performed later on. 
    The idea is to have the dataset reduced to simplify our analysis.
    
    Input: the csv file to prepare and the name of the columns to drop      
    Output: the csv file without the drop columns.
    """
    with open(input_file, "r") as source:
        reader = csv.reader(source)
        with open(output_file, "w", newline='') as result:
            writer = csv.writer(result)
            for row in reader:
                row_count += 1
                print('\r{0}'.format(row_count), end='') # Print rows processed
                for col_index in cols_to_remove:
                    del row[col_index]
                writer.writerow(row)

In [ ]:
#6 handle negative values
def neg_val_treat(df, features):
    """
    This function handles negative values for specific columns where we absolutely
    do not want negative values. In this case, we want to drop the rows that are concerned by
    such values. The features targeted are provided in a list of their names.
    
    Input: the dataframe to prepare and the name of the features to analyse
        
    Output: the dataframe without the rows for which any of the targeted features has
    negative values.
    """
    
    #for each columns listed as an input, we drop the rows that have negative values
    for feature in features:
        neg_condition = df[df[feature] < 0].index
        df = df.drop(neg_condition)

    return df

In [ ]:
#5 calculate trip duration
def trip_duration_calc(pu_time, do_time):
    # computes the duration of the trip in seconds
    trip_duration = do_time - pu_time
    trip_duration = trip_duration/np.timedelta64(1,'s')
    
    return trip_duration

In [ ]:
#3. identify non consistent data
#check consistency between total amount and all fees (yellow and green)
#check consistency regarding datatype

#4 fill in with data
# for numeric or fees => mean of all the rest
# for other => delete the row ?

In [60]:
list_files_green = []
path="data/cleaned"
create_files_list(path,"green",list_files_green)
green_DF = (spark.read
                .option("sep", ",")
                .option("header", True)
                .option("inferSchema", True)
                .csv([list_files_green]))

#print('Proportion of trips witout any passenger')
#ratio = green_DF[green_DF['passenger_count']==0].shape[0]/green_DF.shape[0]*100
#print("{:.2f}".format(ratio),'%')
#


Py4JJavaError: An error occurred while calling o443.csv.
: java.lang.ClassCastException: java.util.ArrayList cannot be cast to java.lang.String
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:42)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:332)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:223)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:211)
	at org.apache.spark.sql.DataFrameReader.csv(DataFrameReader.scala:619)
	at sun.reflect.GeneratedMethodAccessor69.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(Abs

In [ ]:
def handle_no_passenger(df):
    """
    This function splits the dataframe into two dataframes:
    - one with only the trips with no passengers
    - one with all the trips that have at least one passenger

    Input: the dataframe to prepare
        
    Output: two dataframes, one with passengers, and the other one with only the 'empty trips'
    """
    
    #We create two dataframes, one with only the trips with no passengers, the other with passengers
    no_pass_condition = df[df['passenger_count'] == 0].index
    df_full = df.drop(no_pass_condition)
    df_empty = df[df['passenger_count']==0]
    
    return df_empty, df_full

• Yellow taxi records are records that record trip information of New York's famous yellow
taxi cars.

• Green taxi records are records that record trip information by so-called 'boro' taxis a
newer service introduced in August of 2013 to improve taxi service and availability in the
boroughs

• FHV records (short for 'For Hire Vehicles') record information from services that oer
for-hire vehicles (such as Uber, Lyft, Via, and Juno), but also luxury limousine bases.

• High volume FHV (FHVHV for short) are FHV records oered by services that make
more than 10,000 trips per day

## 1. Cleaning the FHV dataset

### Analysis of valid values

|Column Value|Description|Data Type|Constraints|
|---	|---	|---	|---	|
|Dispatching_base_num|License Number of the base that dispatched the trip|String||
|Pickup_datetime|The date and time of the trip pick-up|Datetime|Not Null|
|DropOff_datetime|The date and time of the trip dropoff|Datetime|Not Null|
|PULocationID|Zone in which the trip began|Integer|Not Null|
|DOLocationID|Zone in which the trip ended|Integer|Not Null|
|SR_Flag|Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line); share=1, nonshared=0|Boolean|| 

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: string, PULocationID: double, DOLocationID: string, SR_Flag: string]


DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: string, PULocationID: double, DOLocationID: string, SR_Flag: string]

### Validity rules

1. adjust schema and datatypes
2. remove useless columns 
3. remove rows containing null values for analysis central columns
4. remove duplicate values
5. check locationID consistency

### Identifying dirty records, data repairing

In [262]:
fhvSchema= StructType([
    StructField("Dispatching_base_num",StringType(),True),
    StructField("Pickup_datetime",TimestampType(),True),
    StructField("DropOff_datetime",TimestampType(),True),
    StructField("PULocationID",IntegerType(), True),
    StructField("DOLocationID",IntegerType(),True),
    StructField("SR_Flag",BooleanType(),True)
])

column_list=['pickup_datetime','dropoff_datetime','PULocationID','DOLocationID']
list_files_fhv = []
path="data/cleaned"
create_files_list(path,"fhv",list_files_fhv)
fhv_DF = (spark.read
                .option("sep", ",")
                .option("header", True)
                .schema(userDefinedSchema)
                .csv(list_files_fhv))

print("Number of records before any cleaning step:",(fhv_DF.count(), len(fhv_DF.columns)))

#remove useless columns
fhv_DF= fhv_DF.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID')

#remove null values
fhv_DF= fhv_DF.dropna()

print("Number of records after first cleaning step:",(fhv_DF.count(), len(fhv_DF.columns)))

#remove duplicates
fhv_DF= fhv_DF.dropDuplicates(column_list)

print("Number of records after second cleaning step:",(fhv_DF.count(), len(fhv_DF.columns)))

print("Base schema:")
fhv_DF.printSchema()

zones = gpd.read_file('data/metadata/taxi_zones.shp')
zones_df = spark.createDataFrame(zones).cache()
locID = zones_df.select(col("LocationID"))
locID = [row[0] for row in locID.select("LocationID").collect()]
fhv_DF = fhv_DF.where(col("PULocationid").isin(locID))
fhv_DF = fhv_DF.where(col("DOLocationid").isin(locID)) 
print("Number of records after LocId check and cleaning step:",(fhv_DF.count(), len(fhv_DF.columns)))

Number of records before any cleaning step: (1389608, 6)
Number of records after first cleaning step: (24625, 4)
Number of records after second cleaning step: (24624, 4)
Base schema:
root
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)

Number of records after LocId check and cleaning step: (3184, 4)


## 2. Cleaning the FHVHV dataset

### Analysis of valid values

|Column Value|Description|Data Type|Constraints|
|---	|---	|---	|---	|
|Hvfhs_license_num|TLC license number of the HVFHS base or business|String|Not Null|
|Dispatching_base_num|License Number of the base that dispatched the trip|String|Not Null|
|Pickup_datetime|The date and time of the trip pick-up|Datetime|Not Null|
|DropOff_datetime|The date and time of the trip dropoff|Datetime|Not Null|
|PULocationID|Zone in which the trip began|Integer(smallint)|Not Null|
|DOLocationID|Zone in which the trip ended|Integer(smallint)|Not Null|
|SR_Flag|Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line); share=1, nonshared=0|Boolean|Not Null| 


*Hvfhs_license_num possible values:
• HV0002: Juno
• HV0003: Uber
• HV0004: Via
• HV0005: Lyft

### Validity rules

1. adjust schema and datatypes
2. remove useless columns 
3. remove rows containing null values for analysis central columns
4. remove duplicate values
5. check locationID consistency

### Identifying dirty records, data repairing

In [260]:
fhvhvSchema= StructType([
    StructField("Hvfhs_license_num",StringType(),True),
    StructField("Dispatching_base_num",StringType(),True),
    StructField("Pickup_datetime",TimestampType(),True),
    StructField("DropOff_datetime",TimestampType(),True),
    StructField("PULocationID",IntegerType(), True),
    StructField("DOLocationID",IntegerType(),True),
    StructField("SR_Flag",BooleanType(),True)
])

column_list=['pickup_datetime','dropoff_datetime','PULocationID','DOLocationID']
list_files_fhvhv = []
path="data/cleaned"
create_files_list(path,"fhvhv",list_files_fhvhv)
fhvhv_DF = (spark.read
                .option("sep", ",")
                .option("header", True)
                .schema(fhvhvSchema)
                .csv(list_files_fhvhv))

print("Number of records before any cleaning step:",(fhvhv_DF.count(), len(fhvhv_DF.columns)))

#remove useless columns
fhvhv_DF= fhvhv_DF.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID')

#remove null values
fhvhv_DF= fhvhv_DF.dropna()

print("Number of records after first cleaning step:",(fhvhv_DF.count(), len(fhvhv_DF.columns)))

#remove duplicates
fhvhv_DF= fhvhv_DF.dropDuplicates(column_list)

print("Number of records after second cleaning step:",(fhvhv_DF.count(), len(fhvhv_DF.columns)))

print("Base schema:")
fhvhv_DF.printSchema()

zones = gpd.read_file('data/metadata/taxi_zones.shp')
zones_df = spark.createDataFrame(zones).cache()
locID = zones_df.select(col("LocationID"))
locID = [row[0] for row in locID.select("LocationID").collect()]
fhvhv_DF = fhvhv_DF.where(col("PULocationid").isin(locID))
fhvhv_DF = fhvhv_DF.where(col("DOLocationid").isin(locID)) 
print("Number of records after LocId check and cleaning step:",(fhvhv_DF.count(), len(fhvhv_DF.columns)))

Number of records before any cleaning step: (321819, 7)
Number of records after first cleaning step: (321819, 4)
Number of records after second cleaning step: (321819, 4)
Base schema:
root
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)

Number of records after LocId check and cleaning step: (311874, 4)


## 3. Cleaning the GREEN dataset

### Analysis of valid values

|Column Value|Description|Data Type|Constraints|
|---	|---	|---	|---	|
|VendorID|A code indicating the LPEP provider that provided the record.|Integer(tinyint)|1 or 2, Not Null|
|lpep_pickup_datetime|The date and time when the meter was engaged|Datetime|Not Null|
|lpep_dropoff_datetime|The date and time when the meter was disengaged|Datetime|Not Null|
|Passenger_count|The number of passengers in the vehicle|Integer(tinyint)|???|
|Trip_distance|The elapsed trip distance in miles reported by the taximeter|Decimal|   	|
|PULocationID|Zone in which the taximeter was engaged|Integer(smallint)|Not Null|
|DOLocationID|Zone in which the taximeter was disengaged|Integer(smallint)|Not Null|
|RateCodeID|The final rate code in effect at the end of the trip|Integer(tinyint)|1 to 6, Not Null|
|Store_and_fwd_flag|This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,”because the vehicle did not have a connection to the server|Boolean|Y or N, Not Null|
|Payment_type|A numeric code signifying how the passenger paid for the trip|Integer(tinyint)|1 to 6, Not Null|
|Fare_amount|The time-and-distance fare calculated by the meter|Decimal|Not Null|
|Extra|Miscellaneous extras and surcharges|Decimal|   	|
|MTA_tax|0.50 MTA tax that is automatically triggered based on the metered rate in use|Decimal|   	|
|Improvement_surcharge|0.30 improvement surcharge assessed on hailed trips at the flag drop|Decimal|   	|
|Tip_amount|Tip amount – This field is automatically populated for credit card tips. Cash tips are not included|Decimal|   	|
|Tolls_amount|Total amount of all tolls paid in trip|Decimal|   	|
|Total_amount|The total amount charged to passengers|Decimal|   	|
|Trip_type|A code indicating whether the trip was a street-hail or a dispatch that is automatically assigned based on the metered rate in use but can be altered by the driver|Boolean|1 or 2, Not Null|

### Validity rules

### Identifying dirty records, data repairing

In [270]:
greenSchema= StructType([
        StructField("VendorID",IntegerType(),True),
        StructField("pickup_datetime",TimestampType(),True),
        StructField("dropoff_datetime",TimestampType(),True),
        StructField("Passenger_count",IntegerType(),True),
        StructField("Trip_distance",FloatType(),True),
        StructField("PULocationID",IntegerType(),True),
        StructField("DOLocationID",IntegerType(),True),
        StructField("RateCodeID",IntegerType(),True),
        StructField("Store_and_fwd_flag",BooleanType(),True),
        StructField("Payment_type",IntegerType(),True),
        StructField("Fare_amount",FloatType(),True),
        StructField("Extra",FloatType(),True), 
        StructField("MTA_tax",FloatType(),True),
        StructField("Improvement_surcharge",FloatType(),True),
        StructField("Tip_amount",FloatType(),True),
        StructField("Tolls_amount",FloatType(),True),
        StructField("Total_amount",FloatType(),True),
        StructField("Trip_type",FloatType(),True)
])
                 
column_list=['VendorID','pickup_datetime','dropoff_datetime','Passenger_count','Trip_distance','PULocationID','DOLocationID','RateCodeID','Store_and_fwd_flag','Payment_type','Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount','Trip_type']
list_files_fhvhv = []
path="data/cleaned"
create_files_list(path,"green",list_files_green)
green_DF = (spark.read
                .option("sep", ",")
                .option("header", True)
                .schema(greenSchema)
                .csv(list_files_green))

print("Number of records before any cleaning step:",(green_DF.count(), len(green_DF.columns)))

#remove useless columns
green_DF= green_DF.select('pickup_datetime','dropoff_datetime','Passenger_count','Trip_distance','PULocationID','DOLocationID','Payment_type','Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount','Trip_type')

#remove null values
green_DF= green_DF.dropna()

print("Number of records after first cleaning step:",(green_DF.count(), len(green_DF.columns)))

#remove duplicates
fhvhv_DF= fhvhv_DF.dropDuplicates(column_list)

print("Number of records after second cleaning step:",(green_DF.count(), len(green_DF.columns)))

print("Base schema:")
green_DF.printSchema()

zones = gpd.read_file('data/metadata/taxi_zones.shp')
zones_df = spark.createDataFrame(zones).cache()
locID = zones_df.select(col("LocationID"))
locID = [row[0] for row in locID.select("LocationID").collect()]
green_DF = green_DF.where(col("PULocationid").isin(locID))
green_DF = green_DF.where(col("DOLocationid").isin(locID)) 
print("Number of records after LocId check and cleaning step:",(green_DF.count(), len(green_DF.columns)))

Number of records before any cleaning step: (1595304, 18)
Number of records after first cleaning step: (0, 15)


AnalysisException: 'Cannot resolve column name "Total_amount" among (pickup_datetime, dropoff_datetime, PULocationID, DOLocationID);'

## 4. Cleaning the YELLOW dataset

### Analysis of valid values

|Column Value|Description|Data Type|Constraints|
|---	|---	|---	|---	|
|VendorID|A code indicating the LPEP provider that provided the record.|Integer(tinyint)|1 or 2, Not Null|
|lpep_pickup_datetime|The date and time when the meter was engaged|Datetime|Not Null|
|lpep_dropoff_datetime|The date and time when the meter was disengaged|Datetime|Not Null|
|Passenger_count|The number of passengers in the vehicle|Integer(tinyint)|???|
|Trip_distance|The elapsed trip distance in miles reported by the taximeter|Decimal|   	|
|PULocationID|Zone in which the taximeter was engaged|Integer(smallint)|Not Null|
|DOLocationID|Zone in which the taximeter was disengaged|Integer(smallint)|Not Null|
|RateCodeID|The final rate code in effect at the end of the trip|Integer(tinyint)|1 to 6, Not Null|
|Store_and_fwd_flag|This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,”because the vehicle did not have a connection to the server|Boolean|Y or N, Not Null|
|Payment_type|A numeric code signifying how the passenger paid for the trip|Integer(tinyint)|1 to 6, Not Null|
|Fare_amount|The time-and-distance fare calculated by the meter|Decimal|Not Null|
|Extra|Miscellaneous extras and surcharges|Decimal|   	|
|MTA_tax|0.50 MTA tax that is automatically triggered based on the metered rate in use|Decimal|   	|
|Improvement_surcharge|0.30 improvement surcharge assessed on hailed trips at the flag drop|Decimal|   	|
|Tip_amount|Tip amount – This field is automatically populated for credit card tips. Cash tips are not included|Decimal|   	|
|Tolls_amount|Total amount of all tolls paid in trip|Decimal|   	|
|Total_amount|The total amount charged to passengers|Decimal|   	|

### Validity rules

### Identifying dirty records, data repairing

In [84]:
import pandas as pd
yellow_DF = (spark.read
                .option("sep", ",")
                .option("header", True)
                .option("inferSchema", True)
                .csv("data/cleaned/yellow/*.csv"))